In [35]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, IntSlider, HBox, Label, Button, Layout
from IPython.display import display
from sklearn.metrics import mean_squared_error


np.random.seed(0)

# Generate synthetic data
n_samples = 100  # Increase the number of samples
X = np.sort(np.random.rand(n_samples))
y = np.sin(2 * np.pi * X) + np.random.randn(n_samples) * 0.3

X = X[:, np.newaxis]  # convert to 2D array for sklearn

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

def plot_polynomial_fit(degree, new_test_samples=False):
    if new_test_samples:
        global X_test, y_test
        # Generate new test samples
        X_test_new = np.sort(np.random.rand(y_test.shape[0]))
        y_test_new = np.sin(2 * np.pi * X_test_new) + np.random.randn(y_test.shape[0]) * 0.3
        X_test_new = X_test_new[:, np.newaxis]
        X_test, y_test = X_test_new, y_test_new

    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X_train, y_train)
    
    # Compute the MSE for training and testing data
    train_error = mean_squared_error(y_train, pipeline.predict(X_train))
    test_error = mean_squared_error(y_test, pipeline.predict(X_test))

    X_plot = np.linspace(0, 1, 100)[:, np.newaxis]
    y_plot = pipeline.predict(X_plot)

    plt.figure(figsize=(7, 6))
    plt.plot(X_plot, y_plot, color='red', linewidth=2, label="Model")
    plt.scatter(X_train, y_train, edgecolor='b', s=20, label="Training samples")
    plt.scatter(X_test, y_test, edgecolor='r', s=20, label="Testing samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title(f"Polynomial Regression - Degree {degree}\nTrain error: {train_error:.2f}, Test error: {test_error:.2f}")
    plt.show()

# Create invisible IntSlider
degree_slider = IntSlider(min=1, max=30, value=1, continuous_update=True)

# Create interactive plot and button
interactive_plot = interactive(plot_polynomial_fit, degree=degree_slider, new_test_samples=True)
def on_button_click(_):
    # Increase degree by one and update the plot
    interactive_plot.children[-1].value = not interactive_plot.children[-1].value

underfit_label = Label(value='High Bias (Underfitting)')
overfit_label = Label(value='High Variance (Overfitting)')

# Display everything
display(HBox([underfit_label, Label(value='<---->'), overfit_label]))
display(interactive_plot)

interactive(children=(IntSlider(value=1, description='degree', max=30, min=1), Checkbox(value=True, descriptio…